# 社会经济调查的职业和行业自动编码模型构建

## 数据部分

In [7]:
import pandas as pd

sample1 = pd.read_excel("行业与职业编码数据(一).xlsx")
sample2 = pd.read_excel("行业与职业编码数据(二).xlsx")
code = pd.read_excel("行业与职业分类代码表.xls")

In [16]:
print(sample1)

        受访者编号   题型                                      题目     访员实地填写的答案  类别  \
0      100013  辅助题                                本期自雇单位名称           不知道   0   
1      100013   编码                                您单位主要做什么           不知道  行业   
2      100063  辅助题                                本期自雇单位名称            养鸡   0   
3      100063   编码                                您单位主要做什么            养鸡  行业   
4      100093  辅助题                                  本期单位名称        变压器配件厂   0   
5      100093  辅助题                                  本期部门名称            车间   0   
6      100093  辅助题                                    本期职业             无   0   
7      100093   编码                                您单位主要做什么         变压器配件  行业   
8      100093   编码                               您的工作内容是什么            打杂  职业   
9      100123  辅助题                                本期自雇单位名称           打毛巾   0   
10     100123   编码                                您单位主要做什么           打毛巾  行业   
11     100143  辅助题                      

In [8]:
print(code)

     first  second  third               text  \
0        1       0      0           农、林、牧、渔业   
1        1       1      0                 农业   
2        1       2      0                 林业   
3        1       3      0                畜牧业   
4        1       4      0                 渔业   
5        1       5      0         农、林、牧、渔服务业   
6        2       0      0                采矿业   
7        2       1      0           煤炭开采和洗选业   
8        2       2      0          石油和天然气开采业   
9        2       3      0           黑色金属矿采选业   
10       2       4      0           有色金属矿采选业   
11       2       5      0            非金属矿采选业   
12       2       6      0              其他采矿业   
13       3       0      0                制造业   
14       3       1      0            农副食品加工业   
15       3       2      0              食品制造业   
16       3       3      0              饮料制造业   
17       3       4      0              烟草制品业   
18       3       5      0                纺织业   
19       3       6      0        纺织服装、鞋、

## 接口部分

## 模型部分

In [12]:
import torch
import torch.nn as nn

In [13]:
dir(nn)

['AdaptiveAvgPool1d',
 'AdaptiveAvgPool2d',
 'AdaptiveAvgPool3d',
 'AdaptiveMaxPool1d',
 'AdaptiveMaxPool2d',
 'AdaptiveMaxPool3d',
 'AlphaDropout',
 'AvgPool1d',
 'AvgPool2d',
 'AvgPool3d',
 'BCELoss',
 'BCEWithLogitsLoss',
 'BatchNorm1d',
 'BatchNorm2d',
 'BatchNorm3d',
 'Bilinear',
 'ConstantPad1d',
 'ConstantPad2d',
 'ConstantPad3d',
 'Container',
 'Conv1d',
 'Conv2d',
 'Conv3d',
 'ConvTranspose1d',
 'ConvTranspose2d',
 'ConvTranspose3d',
 'CosineEmbeddingLoss',
 'CosineSimilarity',
 'CrossEntropyLoss',
 'CrossMapLRN2d',
 'DataParallel',
 'Dropout',
 'Dropout2d',
 'Dropout3d',
 'ELU',
 'Embedding',
 'EmbeddingBag',
 'Fold',
 'FractionalMaxPool2d',
 'GLU',
 'GRU',
 'GRUCell',
 'GroupNorm',
 'Hardshrink',
 'Hardtanh',
 'HingeEmbeddingLoss',
 'InstanceNorm1d',
 'InstanceNorm2d',
 'InstanceNorm3d',
 'KLDivLoss',
 'L1Loss',
 'LPPool1d',
 'LPPool2d',
 'LSTM',
 'LSTMCell',
 'LayerNorm',
 'LeakyReLU',
 'Linear',
 'LocalResponseNorm',
 'LogSigmoid',
 'LogSoftmax',
 'MSELoss',
 'MarginRankin

In [15]:
help(nn.BatchNorm2d)

Help on class BatchNorm2d in module torch.nn.modules.batchnorm:

class BatchNorm2d(_BatchNorm)
 |  Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs
 |  with additional channel dimension) as described in the paper
 |  `Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift`_ .
 |  
 |  .. math::
 |  
 |      y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta
 |  
 |  The mean and standard-deviation are calculated per-dimension over
 |  the mini-batches and :math:`\gamma` and :math:`\beta` are learnable parameter vectors
 |  of size `C` (where `C` is the input size).
 |  
 |  By default, during training this layer keeps running estimates of its
 |  computed mean and variance, which are then used for normalization during
 |  evaluation. The running estimates are kept with a default :attr:`momentum`
 |  of 0.1.
 |  
 |  If :attr:`track_running_stats` is set to ``False``, this layer then does not
 |  

In [ ]:
class textCNN(nn.Module):
    def __init__(self,args):
        super(textCNN, self).__init__()
        vocb_size = args['vocb_size']
        dim = args['dim']
        n_class = args['n_class']
        max_len = args['max_len']
        embedding_matrix=args['embedding_matrix']
        # 需要将事先训练好的词向量载入
        self.embeding = nn.Embedding(vocb_size, dim,_weight=embedding_matrix)
        self.conv1 = nn.Sequential(
                     nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5,
                               stride=1, padding=2),
 
                     nn.ReLU(),
                     nn.MaxPool2d(kernel_size=2) # (16,64,64)
                     )
        self.conv2 = nn.Sequential(
                     nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
                     nn.ReLU(),
                     nn.MaxPool2d(2)
                     )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv4 = nn.Sequential(  # (16,64,64)
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.out = nn.Linear(512, n_class)
 
    def forward(self, x):
        x = self.embeding(x)
        x=x.view(x.size(0),1,max_len,word_dim)
        # print(x.size())
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1) # 将（batch，outchanel,w,h）展平为（batch，outchanel*w*h）
        # print(x.size())
        output = self.out(x)
        return output

超参数设置

In [ ]:
LR = 0.001
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
#损失函数
loss_function = nn.CrossEntropyLoss()
#训练批次大小
epoch_size=1000;

训练

In [ ]:
for epoch in range(EPOCH):
 
    for i in range(0,(int)(len(train_x)/epoch_size)):
 
        b_x = Variable(torch.LongTensor(train_x[i*epoch_size:i*epoch_size+epoch_size]))
 
        b_y = Variable(torch.LongTensor((train_y[i*epoch_size:i*epoch_size+epoch_size])))
        output = cnn(b_x)
        loss = loss_function(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(str(i))
        print(loss)
        pred_y = torch.max(output, 1)[1].data.squeeze()
        acc = (b_y == pred_y)
        acc = acc.numpy().sum()
        accuracy = acc / (b_y.size(0))
 
    acc_all = 0;
    for j in range(0, (int)(len(test_x) / test_epoch_size)):
        b_x = Variable(torch.LongTensor(test_x[j * test_epoch_size:j * test_epoch_size + test_epoch_size]))
        b_y = Variable(torch.LongTensor((test_y[j * test_epoch_size:j * test_epoch_size + test_epoch_size])))
        test_output = cnn(b_x)
        pred_y = torch.max(test_output, 1)[1].data.squeeze()
        # print(pred_y)
        # print(test_y)
        acc = (pred_y == b_y)
        acc = acc.numpy().sum()
        print("acc " + str(acc / b_y.size(0)))
        acc_all = acc_all + acc
 
    accuracy = acc_all / (test_y.size(0))
    print("epoch " + str(epoch) + " step " + str(i) + " " + "acc " + str(accuracy))